In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002, 0.003]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [4]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.6287708917339145, 0.6707862165765671)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns

In [6]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 0.1, cols, 816)
single_stump_model['features']

['age_at_current_charge26.0',
 'age_at_current_charge29.0',
 'age_at_current_charge32.0',
 'age_at_current_charge40.0',
 'age_at_current_charge69.0',
 'age_at_current_charge70.0',
 'p_arrest1',
 'p_charges1',
 'p_charges2',
 'p_charges6',
 'p_violence2',
 'p_felony1',
 'p_felony3',
 'p_property2',
 'p_property3',
 'p_misdeassult1',
 'p_drug2',
 'p_drug3',
 'p_dui1',
 'p_stealing1',
 'p_fta_two_year1',
 'p_fta_two_year2',
 'p_probation1',
 'p_incarceration1',
 'six_month1',
 'one_year1',
 'five_year1']

In [7]:
c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [8]:
stumps_summary['best_params'], stumps_summary['KY_score'], stumps_summary['auc_diffs']

({'C': 0.06}, 0.6918390766134408, 0.06820000671053839)

### RiskSLIM

In [9]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['felony_six_month'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['felony_six_month'].values

## columns 
cols = KY_X.columns

In [21]:
single_stump_model = stumps.stump_model(FL_X, FL_Y, KY_X, KY_Y, 
                                        0.04, 
                                        cols, 
                                        816)
selected_features = single_stump_model['features']
len(selected_features)

15

In [22]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [23]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'felony_six_month', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000,
                                max_offset=100,
                                c=1e-6, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/12/19 @ 10:39 PM | 203 rows in lookup table
11/12/19 @ 10:39 PM | ------------------------------------------------------------
11/12/19 @ 10:39 PM | runnning initialization procedure
11/12/19 @ 10:39 PM | ------------------------------------------------------------
11/12/19 @ 10:39 PM | CPA produced 2 cuts
11/12/19 @ 10:39 PM | running naive rounding on 4 solutions
11/12/19 @ 10:39 PM | best objective value: 0.6931
11/12/19 @ 10:39 PM | rounding produced 4 integer solutions
11/12/19 @ 10:39 PM | best objective value is 0.6931
11/12/19 @ 10:39 PM | running sequential rounding on 4 solutions
11/12/19 @ 10:39 PM | best objective value: 0.6931
11/12/19 @ 10:39 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 10:39 PM | polishing 4 solutions
11/12/19 @ 10:39 PM | best objective value: 0.6931
11/12/19 @ 10:39 PM | polishing produced 4 integer solutions
11/12/19 @ 10:39 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:39 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2979.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 199220 31004        0.2902     3        0.2938        0.2865   627446    2.48%           rho_5 D 199220 199219     37
 203693 31337        0.2916     4        0.2938        0.2866   639188    2.46%           rho_4 D 203693 203692     25
Elapsed time = 25.11 sec. (41196.25 ticks, tree = 8.26 MB, solutions = 9)
 208286 31486        0.2867     4        0.2938        0.2867   651105    2.43%           rho_5 U 208286 208283     24
 212660 31635        0.2920     1        0.2938        0.2867   662810    2.42%           rho_4 U 212660 212659     35
 217105 31661        0.2900     1        0.2938        0.2868   674817    2.40%           rho_5 D 217105 217103     31
 221512 32064        cutoff              0.2938        0.2868   687054    2.38%           rho_4 D 221512 221511     26
 226491 32432        0.2875     4        0.2938        0.2868   695196    2.38%           rho_4 U 226491 226489     37
 231388 32934        cutoff              0.2938        0.2868   704198    2.37%           rho

 675798 54233        0.2893     9        0.2938        0.2893  1780200    1.52%           rho_3 D 675798 675792     24
 694512 56680        0.2894     5        0.2938        0.2894  1820437    1.49%           rho_2 D 694512 694511     24
 713206 57189        cutoff              0.2938        0.2895  1862377    1.47%           rho_2 D 713206 713205     31
 731031 57100        0.2914     1        0.2938        0.2896  1903928    1.44%           rho_0 D 731031 731029     42
 748633 55766        0.2924     1        0.2938        0.2897  1943666    1.40%           rho_3 D 748633 748631     36
 766295 56067        0.2898     7        0.2938        0.2898  1988104    1.37%          rho_12 U 766295 766294     22
 785430 55844        0.2914     5        0.2938        0.2899  2026960    1.34%           rho_4 D 785430 785429     35
 804428 56423        cutoff              0.2938        0.2899  2069786    1.32%           rho_2 U 804428   3077     32
Elapsed time = 86.09 sec. (168045.89 ticks, tree

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:41 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3022.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

 289320 44080        0.2978     7        0.2983        0.2935  1159843    1.61%           rho_4 D 289320 289319     36
 295433 44189        0.2942     6        0.2983        0.2937  1178080    1.55%           rho_4 U 295433 295431     37
 301803 44017        0.2938     1        0.2983        0.2938  1194806    1.52%           rho_4 U 301803 273246     37
 308407 44506        0.2979     5        0.2983        0.2939  1212881    1.49%          rho_11 U 308407 308405     31
 314489 44418        0.2968     7        0.2983        0.2940  1231518    1.46%           rho_3 N 314489 308653     20
 321095 44612        0.2941     2        0.2983        0.2941  1247175    1.43%           rho_0 U 321095 321094     45
 327668 44318        0.2948     1        0.2983        0.2941  1261194    1.42%           rho_2 U 327668 327527     22
 333652 43684        cutoff              0.2983        0.2941  1279168    1.40%          rho_15 D 333652  61339     30
 339236 42541        0.2951     6        0.2983 

11/12/19 @ 10:51 PM | ------------------------------------------------------------
11/12/19 @ 10:51 PM | runnning initialization procedure
11/12/19 @ 10:51 PM | ------------------------------------------------------------
11/12/19 @ 10:51 PM | CPA produced 2 cuts
11/12/19 @ 10:51 PM | running naive rounding on 5 solutions
11/12/19 @ 10:51 PM | best objective value: 0.6931
11/12/19 @ 10:51 PM | rounding produced 5 integer solutions
11/12/19 @ 10:51 PM | best objective value is 0.6931
11/12/19 @ 10:51 PM | running sequential rounding on 5 solutions
11/12/19 @ 10:51 PM | best objective value: 0.6931
11/12/19 @ 10:51 PM | sequential rounding produced 0 integer solutions
11/12/19 @ 10:51 PM | polishing 5 solutions
11/12/19 @ 10:51 PM | best objective value: 0.6931
11/12/19 @ 10:51 PM | polishing produced 5 integer solutions
11/12/19 @ 10:51 PM | initialization produced 9 feasible solutions
11/12/19 @ 10:51 PM | best objective value: 0.3005
11/12/19 @ 10:51 PM | -----------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/12/19 @ 10:51 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3005.
Tried aggregator 1 time.
Reduced MIP has 32 rows, 34 columns, and 93 nonzeros.
Reduced MIP has 15 binaries, 17 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 194249 48183        0.2936     3        0.2938        0.2859  1065926    2.68%          rho_11 U 194249 194248     41
 198400 49553        0.2912     4        0.2938        0.2859  1088809    2.68%          rho_11 U 198400 198398     31
 202866 50821        cutoff              0.2938        0.2859  1110871    2.68%           rho_7 U 202866 202865     40
 207395 52310        0.2889     6        0.2938        0.2859  1133327    2.68%          rho_11 U 207395 207394     29
 212009 53770        0.2859     7        0.2938        0.2859  1155919    2.68%           rho_3 U 212009 212005     39
 216814 54843        0.2880     1        0.2938        0.2859  1175202    2.68%           rho_0 U 216814 216813     31
 220852 55715        0.2878    10        0.2938        0.2859  1194577    2.68%           rho_5 U 220852 220851     21
 224523 56003        0.2915     9        0.2938        0.2859  1215268    2.68%          rho_10 U 224523 224522     40
 228192 56499        cutoff              0.2938 

 687010 122938        0.2922    12        0.2938        0.2885  2794577    1.80%           rho_4 N 687010 687009     33
 704796 122658        cutoff              0.2938        0.2886  2851730    1.77%           rho_6 U 704796 590329     46
 721089 121935        0.2905     9        0.2938        0.2887  2914682    1.73%          rho_12 U 721089 721087     47
 738130 122294        0.2893     8        0.2938        0.2888  2968409    1.70%           rho_4 U 738130 738128     39
 754260 120756        0.2934     1        0.2938        0.2889  3026330    1.66%           rho_4 U 754260 754259     37
 771250 120780        0.2932     7        0.2938        0.2890  3083095    1.63%          rho_14 D 771250 771249     40
Elapsed time = 102.41 sec. (167310.19 ticks, tree = 31.85 MB, solutions = 6)
 789050 119304        0.2896     9        0.2938        0.2891  3140641    1.59%           rho_4 N 789050 338191     34
 806446 119361        cutoff              0.2938        0.2892  3195515    1.56%   

In [24]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_validation_score'])

(0.5963508164135645, 0.5920631598258452)

In [25]:
#### save results
summary_F6_fl_inter_model = {"CART": cart_summary,
                             "EBM": ebm_summary,
                             "Stumps": stumps_summary,
                             "RiskSLIM": riskslim_summary}

In [26]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Six Month\\"
results = [["Felony",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Six Month FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)